# Encontro 13: Medidas de Centralidade

Importando a biblioteca:

In [1]:
import sys
sys.path.append('..')

from random import choice
from itertools import permutations

import pandas as pd
import networkx as nx

import socnet as sn

Configurando a biblioteca:

In [2]:
sn.node_size = 10
sn.node_color = (255, 255, 255)

sn.edge_width = 1
sn.edge_color = (192, 192, 192)

sn.node_label_position = 'top center'

Carregando rede de casamentos entre famílias de Florença durante a Renascença.

J. F. Padgett e C. K. Ansell. *Robust action and the rise of the Medici, 1400–1434.* American Journal of
Sociology 98, págs. 1259-1319, 1993.

In [3]:
g = sn.load_graph('Renaissance.gml', has_pos=True)

sn.show_graph(g, nlab=True)

Função que registra, em cada nó, seus sucessores em geodésicas de $s$ a $t$.

In [4]:
def set_geodesic_successors(g, s, t):
    for n in g.nodes:
        g.nodes[n]['geodesic_successors'] = set()

    for p in nx.all_shortest_paths(g, s, t):
        for i in range(len(p) - 1):
            g.nodes[p[i]]['geodesic_successors'].add(p[i + 1])

Funções que representam uma escolha aleatória de sucessor para diferentes tipos de trajetórias.

In [5]:
# Pense que o atributo 'passages' abaixo indica quantas
# vezes um fluxo já passou por um nó ou por uma aresta.

def random_geodesic_successor(g, n):
    return choice([m for m in g.nodes[n]['geodesic_successors']])

def random_path_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.nodes[m]['passages'] == 0])

def random_trail_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.edges[n, m]['passages'] == 0])

def random_walk_successor(g, n):
    return choice([m for m in g.neighbors(n)])

Função que faz uma simulação de fluxo de $s$ a $t$, que pode ou não ser bem-sucedida.

In [6]:
def simulate_single_flow(g, s, t, func_traj, difusao):
    # Inicializa o atributo 'passages' de cada nó.
    for n in g.nodes:
        g.nodes[n]['passages'] = 0
    g.nodes[s]['passages'] = 1

    # Inicializa o atributo 'passages' de cada aresta.
    for n, m in g.edges:
        g.edges[n, m]['passages'] = 0

    # Inicializa s como o único dono do insumo.
    for n in g.nodes:
        g.nodes[n]['owner'] = False
    g.nodes[s]['owner'] = True

    # Simula o fluxo, contando o número total de passos.

    steps = 0

    while True:
        # O conjunto reached representa todos os nós
        # que o fluxo consegue alcançar no passo atual.
        reached = set()

        # Verifica cada um dos donos atuais do insumo.

        owners = [n for n in g.nodes if g.nodes[n]['owner']]

        for n in owners:
            # Deixa de ser dono do insumo.
            g.nodes[n]['owner'] = difusao ###
            
            try:
                # Escolhe aleatoriamente um dos sucessores.
                m = func_traj(g, n) ##
            except IndexError:
                continue
            
            # Deixa de ser dono do insumo.
            g.nodes[n]['owner'] = difusao ###
            
            # Incrementa o atributo 'passages' do nó.
            g.nodes[m]['passages'] += 1

            # Incrementa o atributo 'passages' da aresta.
            g.edges[n, m]['passages'] += 1

            # Registra que consegue alcançar esse nó.
            reached.add(m)

            
        # Todo nó alcançado passa a ser dono do insumo.

        for n in reached:
            g.nodes[n]['owner'] = True

        # Isso conclui o passo atual da simulação.
        steps += 1

        # Se o passo alcançou t, chegamos ao fim da simulação.
        # Ela foi bem-sucedida: devolvemos o número de passos.
        if t in reached:
            return steps

        # Se o passo não alcançou ninguém, chegamos ao fim da
        # simulação. Ela não foi bem-sucedida: devolvemos -1.
        if not reached:
            return -1

Função que faz simulações de fluxo de $s$ a $t$ até uma ser bem-sucedida.

In [7]:
def simulate_successful_flow(g, s, t, func_traj, difusao):
    set_geodesic_successors(g, s, t)

    while True:
        steps = simulate_single_flow(g, s, t, func_traj, difusao)

        if steps != -1:
            return steps

Função que faz simulações de fluxo para todo $s$ e $t$ possíveis, e tira disso um *closeness simulado* e um *betweenness simulado*.

In [8]:
def simulate_all_flows(g,func_traj, difusao):
    for n in g.nodes:
        g.nodes[n]['closeness'] = 0
        g.nodes[n]['betweenness'] = 0

    for s, t in permutations(g.nodes, 2):
        steps = simulate_successful_flow(g, s, t, func_traj, difusao)

        g.nodes[s]['closeness'] += steps
        for n in g.nodes:
            if n != s and n != t:
                g.nodes[n]['betweenness'] += g.nodes[n]['passages']

    # Normalizações necessárias para comparar com os
    # resultados analíticos. Não precisa entender.
    for n in g.nodes:
        g.nodes[n]['closeness'] = (g.number_of_nodes() - 1) / g.nodes[n]['closeness']
        g.nodes[n]['betweenness'] /= (g.number_of_nodes() - 1) * (g.number_of_nodes() - 2)

Média de *closeness simulado* e *betweenness simulado* para muitas repetições da simulação acima.

In [9]:
# TIMES = 1000
# func_traj = random_geodesic_successor
# difusao = False

def simul(TIMES, func_traj, difusao):
    for n in g.nodes:
        g.nodes[n]['mean_closeness'] = 0
        g.nodes[n]['mean_betweenness'] = 0

    for _ in range(TIMES):
        simulate_all_flows(g, func_traj, difusao)

        for n in g.nodes:
            g.nodes[n]['mean_closeness'] += g.nodes[n]['closeness']
            g.nodes[n]['mean_betweenness'] += g.nodes[n]['betweenness']

    for n in g.nodes:
        g.nodes[n]['mean_closeness'] /= TIMES
        g.nodes[n]['mean_betweenness'] /= TIMES
        
    df = pd.DataFrame({
    'família': [g.nodes[n]['label'] for n in g.nodes],
    'closeness simulado': [g.nodes[n]['mean_closeness'] for n in g.nodes],
    'closeness analítico': [cc[n] for n in g.nodes],
    'betweenness simulado': [g.nodes[n]['mean_betweenness'] for n in g.nodes],
    'betweenness analítico': [bc[n] for n in g.nodes],
    })
    return df
#     return df.sort_values(coluna, ascending=0)#.describe()

Cálculo de *closeness* e *betweenness* a partir das funções prontas da NetworkX, para comparação.

In [10]:
cc = nx.closeness_centrality(g)

bc = nx.betweenness_centrality(g)

Construção de data frame só para comparar mais facilmente.

E agora, vamos pensar um pouco...

* Onde você precisa mudar o código para usar uma *trajetória* que não seja a *geodésica*? (caminho, trilha, passeio)

* Onde você precisa mudar o código para usar uma *difusão* que não seja a *transferência*? (duplicação)

Considere então a seguinte **hipótese**:

>Quando consideramos outros tipos de trajetória e outros tipos de difusão, os nós com maior *closeness simulado* e *betweenness simulado* não são necessariamente os nós com maior *closeness* e *betweenness* segundo as fórmulas clássicas. (que correspondem ao uso de geodésica e transferência na simulação)

Queremos:

1. Operacionalização e teste dessas hipótese. (Objetivo 3)
2. Interpretação dos resultados na linguagem de Análise de Redes Sociais (Objetivo 4)

Um *feedback* da atividade sobre *coreness no Jazz* será dado em breve, para vocês terem uma melhor referência do item 2.

In [11]:
from scipy import stats

In [35]:
bm = simul(100, random_geodesic_successor, False)
bmc = bm["closeness simulado"]
bmb = bm["betweenness simulado"]
bm.describe()

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado
count,15.000000,15.000000,15.000000,15.000000
mean,0.115751,0.115751,0.411834,0.411834
std,0.136863,0.136257,0.074293,0.074293
min,0.000000,0.000000,0.285714,0.285714
25%,0.010989,0.011154,0.359211,0.359211
50%,0.087912,0.089066,0.400000,0.400000
75%,0.131868,0.131401,0.474713,0.474713
max,0.521978,0.521758,0.560000,0.560000


In [36]:
gd = simul(100, random_geodesic_successor, True)
gdc = gd["closeness simulado"]
gdb = gd["betweenness simulado"]
gd.describe()

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado
count,15.000000,15.000000,15.000000,15.000000
mean,0.115751,0.311813,0.411834,0.411834
std,0.136863,0.364757,0.074293,0.074293
min,0.000000,0.000000,0.285714,0.285714
25%,0.010989,0.027555,0.359211,0.359211
50%,0.087912,0.242308,0.400000,0.400000
75%,0.131868,0.362527,0.474713,0.474713
max,0.521978,1.390110,0.560000,0.560000


In [14]:
stats.ttest_ind(bmc,gdc)

Ttest_indResult(statistic=0.0, pvalue=1.0)

In [15]:
stats.ttest_ind(bmb,gdb)

Ttest_indResult(statistic=-1.9496608670291715, pvalue=0.061295876755952494)

In [37]:
pt = simul(100, random_path_successor, False)
ptc = pt["closeness simulado"]
ptb = pt["betweenness simulado"]
pt.describe()

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado
count,15.000000,15.000000,15.000000,15.000000
mean,0.115751,0.246081,0.411834,0.244629
std,0.136863,0.187153,0.074293,0.027429
min,0.000000,0.000000,0.285714,0.197019
25%,0.010989,0.071429,0.359211,0.231727
50%,0.087912,0.246209,0.400000,0.248204
75%,0.131868,0.365000,0.474713,0.261702
max,0.521978,0.628956,0.560000,0.304239


In [17]:
stats.ttest_ind(bmc,ptc)

Ttest_indResult(statistic=8.2819649337103591, pvalue=5.1783110742188954e-09)

In [18]:
stats.ttest_ind(bmb,ptb)

Ttest_indResult(statistic=-2.2089736954764083, pvalue=0.035528542837842826)

In [38]:
pd1 = simul(100, random_path_successor, True)
pdc = pd1["closeness simulado"]
pdb = pd1["betweenness simulado"]
pd1.describe()

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado
count,15.000000,15.000000,15.000000,15.000000
mean,0.115751,0.565982,0.411834,0.251839
std,0.136863,0.157660,0.074293,0.021288
min,0.000000,0.261923,0.285714,0.204285
25%,0.010989,0.419918,0.359211,0.234839
50%,0.087912,0.613407,0.400000,0.257610
75%,0.131868,0.678791,0.474713,0.265166
max,0.521978,0.819505,0.560000,0.281031


In [20]:
stats.ttest_ind(bmc,pdc)

Ttest_indResult(statistic=8.0541912671476155, pvalue=9.0428211170748702e-09)

In [21]:
stats.ttest_ind(bmb,pdb)

Ttest_indResult(statistic=-8.3211520017744416, pvalue=4.7079339187735153e-09)

In [39]:
tt = simul(100, random_trail_successor, False)
ttc = tt["closeness simulado"]
ttb = tt["betweenness simulado"]
tt.describe()

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado
count,15.000000,15.000000,15.000000,15.000000
mean,0.115751,0.247487,0.411834,0.239996
std,0.136863,0.195281,0.074293,0.025891
min,0.000000,0.000000,0.285714,0.195206
25%,0.010989,0.071429,0.359211,0.229062
50%,0.087912,0.268901,0.400000,0.237285
75%,0.131868,0.340495,0.474713,0.255201
max,0.521978,0.704121,0.560000,0.296026


In [27]:
stats.ttest_ind(bmc,ttc)

Ttest_indResult(statistic=8.4531522814476414, pvalue=3.421071847582356e-09)

In [28]:
stats.ttest_ind(bmb,ttb)

Ttest_indResult(statistic=-2.1349864952179685, pvalue=0.041652113185942856)

In [40]:
td = simul(100, random_trail_successor, True)
tdc = td["closeness simulado"]
tdb = td["betweenness simulado"]
td.describe()

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado
count,15.000000,15.000000,15.000000,15.000000
mean,0.115751,0.682725,0.411834,0.240704
std,0.136863,0.281287,0.074293,0.018674
min,0.000000,0.261264,0.285714,0.199611
25%,0.010989,0.407610,0.359211,0.227659
50%,0.087912,0.689780,0.400000,0.246930
75%,0.131868,0.897555,0.474713,0.253073
max,0.521978,1.252527,0.560000,0.264948


In [29]:
stats.ttest_ind(bmc,tdc)

Ttest_indResult(statistic=8.6582899496610715, pvalue=2.0923746932200899e-09)

In [30]:
stats.ttest_ind(bmb,tdb)

Ttest_indResult(statistic=-7.0594296798597531, pvalue=1.1148524412269425e-07)

In [41]:
wt = simul(100, random_walk_successor, False)
wtc = wt["closeness simulado"]
wtb = wt["betweenness simulado"]
wt.describe()

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado
count,15.000000,15.000000,15.000000,15.000000
mean,0.115751,2.031491,0.411834,0.036688
std,0.136863,1.148918,0.074293,0.002007
min,0.000000,0.701319,0.285714,0.033860
25%,0.010989,1.149341,0.359211,0.034842
50%,0.087912,2.405879,0.400000,0.037269
75%,0.131868,2.435385,0.474713,0.037906
max,0.521978,4.980604,0.560000,0.039710


In [31]:
stats.ttest_ind(bmc,wtc)

Ttest_indResult(statistic=19.545370760047543, pvalue=7.4081179495548969e-18)

In [32]:
stats.ttest_ind(bmb,wtb)

Ttest_indResult(statistic=-6.4172475825349569, pvalue=5.9999622184773337e-07)

In [42]:
wd = simul(100, random_walk_successor, True)
wdc = wd["closeness simulado"]
wdb = wd["betweenness simulado"]
wd.describe()

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado
count,15.000000,15.000000,15.000000,15.000000
mean,0.115751,2.340242,0.411834,0.156064
std,0.136863,1.636734,0.074293,0.012455
min,0.000000,0.569231,0.285714,0.128066
25%,0.010989,1.117967,0.359211,0.147441
50%,0.087912,2.375769,0.400000,0.156715
75%,0.131868,2.825055,0.474713,0.165443
max,0.521978,6.792692,0.560000,0.172205


In [33]:
stats.ttest_ind(bmc,wdc)

Ttest_indResult(statistic=13.208101228283933, pvalue=1.5002264486240357e-13)

In [34]:
stats.ttest_ind(bmc,wdb)

Ttest_indResult(statistic=-4.5802439110755504, pvalue=8.731333655857842e-05)

A hiopotese nula (as duas amostras sao iguais) é rejeitada quando o p-value é menor que 0.05.

Dessa forma, a única situação onde não há rejeição da hipótese nula (tanto no closeness quanto no betweeness) é no caso de geodésica com difusão, uma vez que seu p-valor é maior que 5%.